In [1]:
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.compute as pc
import pyarrow.dataset as ds
import h5py
from PIL import Image
from src.benchmarkers import *
from src.benchmarkersV2 import *
from tqdm import tqdm
import os
import numpy as np
from io import BytesIO
from typing import *
#import framework_functions as ff

In [2]:
#table = pq.read_table(r"C:\Users\Cristiano Lavoro\Downloads\ds_10.parquet")
table = pq.read_table(r"C:\Users\Cristiano Lavoro\Desktop\benchmarks\imagenette\imagenette2\output\ds.parquet")

In [3]:
print(ff.draw_tree_schema(table.take([0]).to_pydict()))

└── image_feature
    └── [0]
        └── [0]
            ├── class_feature
            │   ├── [0]
            │   │   └── label
            ├── image
            └── text_feature
                └── [0]
                    └── text



In [18]:
schema = table.schema
batches = [recordbatch for recordbatch in table.to_batches(300) if len(recordbatch) > 0]
offsets = np.cumsum([0] + [len(b) for b in batches], dtype=np.int64)

In [11]:
index = 320
length = 1

In [3]:
def interpolation_search(arr: List[int], x: int) -> int:
    """
    Return the position i of a sorted array so that arr[i] <= x < arr[i+1]

    Args:
        arr (`List[int]`): non-empty sorted list of integers
        x (`int`): query

    Returns:
        `int`: the position i so that arr[i] <= x < arr[i+1]

    Raises:
        `IndexError`: if the array is empty or if the query is outside the array values
    """
    i, j = 0, len(arr) - 1
    while i < j and arr[i] <= x < arr[j]:
        k = i + ((j - i) * (x - arr[i]) // (arr[j] - arr[i]))
        if arr[k] <= x < arr[k + 1]:
            return k
        elif arr[k] < x:
            i, j = k + 1, j
        else:
            i, j = i, k
    raise IndexError(f"Invalid query '{x}' for size {arr[-1] if len(arr) else 'none'}.")

In [ ]:
### Non ci interessa fare lo sclicing di un vettore di batch che, di fatto, è già in memory. Dato il vettore di 
### offset (creato e storato come metadato in fase di scrittura partizionata) dobbiamo aprire on the fly la tabella (o il batch, che dir si voglia)
### dato l'indice e l'informazione storata nell'offset

In [15]:
def stolen(index,batches):
    offset=index
    if offset < 0:
        raise IndexError("Offset must be non-negative")
    elif offset >= offsets[-1] or (length is not None and length <= 0):
        return pa.Table.from_batches([], schema=schema)
    i = _interpolation_search(offsets, offset)
    if length is None or length + offset >= offsets[-1]:
        batches = batches[i:]
        batches[0] = batches[0].slice(offset - offsets[i])
    else:
        j = _interpolation_search(offsets, offset + length - 1)
        batches = batches[i : j + 1]
        batches[-1] = batches[-1].slice(0, offset + length - offsets[j])
        batches[0] = batches[0].slice(offset - offsets[i])
    return pa.Table.from_batches(batches, schema=schema)

In [4]:
ff.partition_dataset(table,"imagenette/imagenette2/output/partitioning")

In [34]:
offsets = np.load("imagenette/imagenette2/output/partitioning/ds_offsets.npy")
index_file_to_open = _interpolation_search(offsets,800)
#with pa.memory_map(f'{path}_{item}_stream.arrows', 'rb') as source:
#                        table = pa.ipc.open_stream(source).read_all()
#                        en_time_load = time.time()
#                        tmp_load.append(en_time_load - st_time_load)
print(len(offsets))

15


In [38]:
list(range(2,len(offsets)))

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

In [7]:
root_path = "imagenette/imagenette2/output/partitioning"

In [41]:
with pa.memory_map(f"{root_path}/ds_{1}_of_{len(offsets)-1}_stream.arrows", 'rb') as source:
        table = pa.ipc.open_stream(source).read_all()

for index_file_to_open in list(range(2,len(offsets))):
        with pa.memory_map(f"{root_path}/ds_{index_file_to_open}_of_{len(offsets)-1}_stream.arrows", 'rb') as source:
                table = pa.concat_tables([table,pa.ipc.open_stream(source).read_all()])

In [30]:
len(table.take([800-offsets[index_file_to_open]]))

1

In [29]:
800-offsets[index_file_to_open]

200

In [6]:
TEST_paths = [["image_feature","class_feature","label"]]

In [13]:
def get_sample(root_path : str,
               sample_index : int,
               all_data_in_memory : bool = False,
               memory_map : bool = False,
               mode : str = "stream"
               ):
    
        offsets = np.load(f"{root_path}/ds_offsets.npy")

        #### ALL DATA IN MEMORY ####

        if all_data_in_memory and mode=="stream" and memory_map:
                with pa.memory_map(f"{root_path}/ds_{1}_of_{len(offsets)-1}_stream.arrows", 'rb') as source:
                        table = pa.ipc.open_stream(source).read_all()
                for i in list(range(2,len(offsets))):
                        with pa.memory_map(f"{root_path}/ds_{i}_of_{len(offsets)-1}_stream.arrows", 'rb') as source:
                                table = pa.concat_tables([table,pa.ipc.open_stream(source).read_all()])
                table = table.combine_chunks()
        
        if all_data_in_memory and mode=="stream" and not memory_map:
                with pa.OSFile(f"{root_path}/ds_{1}_of_{len(offsets)-1}_stream.arrows", 'rb') as source:
                        table = pa.ipc.open_stream(source).read_all()
                for i in list(range(2,len(offsets))):
                        with pa.OSFile(f"{root_path}/ds_{i}_of_{len(offsets)-1}_stream.arrows", 'rb') as source:
                                table = pa.concat_tables([table,pa.ipc.open_stream(source).read_all()])
                table = table.combine_chunks()

        if all_data_in_memory and mode=="file" and memory_map:
                with pa.memory_map(f"{root_path}/ds_{1}_of_{len(offsets)-1}_file.arrow", 'rb') as source:
                        table = pa.ipc.open_file(source).read_all()
                for i in list(range(2,len(offsets))):
                        with pa.memory_map(f"{root_path}/ds_{i}_of_{len(offsets)-1}_file.arrow", 'rb') as source:
                                table = pa.concat_tables([table,pa.ipc.open_file(source).read_all()])
                table = table.combine_chunks()

        if all_data_in_memory and mode=="file" and not memory_map:
                with pa.OSFile(f"{root_path}/ds_{1}_of_{len(offsets)-1}_file.arrow", 'rb') as source:
                        table = pa.ipc.open_file(source).read_all()
                for i in list(range(2,len(offsets))):
                        with pa.OSFile(f"{root_path}/ds_{i}_of_{len(offsets)-1}_file.arrow", 'rb') as source:
                                table = pa.concat_tables([table,pa.ipc.open_file(source).read_all()])
                table = table.combine_chunks()

        #### PARTITIONING ####

        index_file_to_open = interpolation_search(offsets,sample_index)

        if not all_data_in_memory and mode=="stream" and memory_map:
                with pa.memory_map(f"{root_path}/ds_{index_file_to_open+1}_of_{len(offsets)-1}_stream.arrows", 'rb') as source:
                        table = pa.ipc.open_stream(source).read_all()
        
        if not all_data_in_memory and mode=="stream" and not memory_map:
                with pa.OSFile(f"{root_path}/ds_{index_file_to_open+1}_of_{len(offsets)-1}_stream.arrows", 'rb') as source:
                        table = pa.ipc.open_stream(source).read_all()
               
        if not all_data_in_memory and mode=="file" and memory_map:
                with pa.memory_map(f"{root_path}/ds_{index_file_to_open+1}_of_{len(offsets)-1}_file.arrow", 'rb') as source:
                        table = pa.ipc.open_file(source).read_all()
               

        if not all_data_in_memory and mode=="file" and not memory_map:
                with pa.OSFile(f"{root_path}/ds_{index_file_to_open+1}_of_{len(offsets)-1}_file.arrow", 'rb') as source:
                        table = pa.ipc.open_file(source).read_all()

        if not all_data_in_memory:
                sample_index = sample_index-offsets[index_file_to_open]

        
        return get_sample_features(table,sample_index,TEST_paths)

In [9]:
def get_sample_features(table : pa.Table,
                        sample_index : int,
                        feature_list_paths: List[List[str]],
                        feature_list_indexes : List[List[List[int]]] = None,
                ):

    sample = {}
    if feature_list_indexes:
      feature_list_indexes = [item.insert(0,[sample_index]) for item in feature_list_indexes]
      for path,index in zip(feature_list_paths,feature_list_indexes):
        obj = None
        for i, feature_name in enumerate(path):
            if i == 0:
                obj = table.column(feature_name).chunk(0).take(index[i]) 
            else:
                obj = obj.values.field(feature_name).take(index[i])
        sample[str(path[-1])] = obj.to_pylist()

    elif not feature_list_indexes:
      for path in feature_list_paths :
        obj = None
        for i, feature_name in enumerate(path):
            if i == 0:
                obj = table.column(feature_name).chunk(0).take([sample_index])   
            else:
                obj = obj.values.field(feature_name)
        sample[str(path[-1])] = obj.to_pylist()
    else:
       raise TypeError("either list of indices or single index must be provided")
    return sample

In [16]:
print(get_sample(root_path,1,all_data_in_memory=False))

{'label': [0]}
